In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [ ]:
import cv2
from PIL import  Image
import numpy as np

In [ ]:
image_directory = '/data1/hom1/ict13/Coffee Plant Dataset/'

dataset=[]
label=[]

Cerscospora = os.listdir('/data1/hom1/ict13/Coffee Plant Dataset/Cerscospora')
Healthy = os.listdir('/data1/hom1/ict13/Coffee Plant Dataset/Healthy')
Miner = os.listdir('/data1/hom1/ict13/Coffee Plant Dataset/Miner')
Phoma = os.listdir('/data1/hom1/ict13/Coffee Plant Dataset/Phoma')
Rust = os.listdir('/data1/hom1/ict13/Coffee Plant Dataset/Rust')

In [ ]:
for i , image_name in enumerate(Cerscospora):
  if(image_name.split('.')[1]== 'jpg'):
          image=cv2.imread(os.path.join(image_directory, 'Cerscospora/', image_name))
          image=Image.fromarray(image, 'RGB')
          image=image.resize((224,224))
          dataset.append(np.array(image))
          label.append(0)

In [ ]:
for i , image_name in enumerate(Healthy):
  if(image_name.split('.')[1]== 'jpg'):
      image=cv2.imread(os.path.join(image_directory, 'Healthy/', image_name))
      image=Image.fromarray(image, 'RGB')
      image=image.resize((224,224))
      dataset.append(np.array(image))
      label.append(1)

In [ ]:
for i , image_name in enumerate(Miner):
  if(image_name.split('.')[1]== 'jpg'):
      image=cv2.imread(os.path.join(image_directory,'Miner/',image_name))
      image=Image.fromarray(image, 'RGB')
      image=image.resize((224,224))
      dataset.append(np.array(image))
      label.append(2)

In [ ]:
for i , image_name in enumerate(Phoma):
  if(image_name.split('.')[1]== 'jpg'):
      image=cv2.imread(os.path.join(image_directory, 'Phoma/', image_name))
      image=Image.fromarray(image, 'RGB')
      image=image.resize((224,224))
      dataset.append(np.array(image))
      label.append(3)

In [ ]:
for i , image_name in enumerate(Rust):
  if(image_name.split('.')[1]== 'jpg'):
      image=cv2.imread(os.path.join(image_directory, 'Rust/', image_name))
      image=Image.fromarray(image, 'RGB')
      image=image.resize((224,224))
      dataset.append(np.array(image))
      label.append(4)

In [ ]:
print(len(dataset))
print(len(label))
dataset=np.array(dataset)
label=np.array(label)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.utils.np_utils import normalize
x_train= normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
inc=InceptionResNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


for i in inc.layers:
  i.trainable=False

x=Flatten()(inc.output)
prediction=Dense(5, activation='softmax')(x)

In [ ]:
from tensorflow.keras.models import Model
model=Model(inputs=inc.input, outputs=prediction)

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

learn_rate=.005

sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)

In [ ]:
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist=model.fit(x_train, y_train, batch_size=32, 
               verbose=1, epochs=50, 
               validation_data=(x_test,y_test), shuffle=False)

model.save('/data1/hom1/ict13/Coffee Plant Dataset/Models/vgg_coffee.h5')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[['accuracy', 'val_accuracy']].plot()
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

In [ ]:
metrics[['loss', 'val_loss']].plot()
plt.title("Model Loss")
plt.ylabel("loss")
plt.xlabel("Epoch")